In [360]:
## Librerias que usare para el analisis
import pandas as pd
import altair as alt
import numpy as np

In [361]:
#Librerias para la prediccion

from sklearn.model_selection import train_test_split #separar datos
from sklearn.preprocessing import StandardScaler #escalador
from sklearn.linear_model import LogisticRegression #reg logistica
from sklearn.metrics import confusion_matrix #matriz de confusion
from sklearn.metrics import accuracy_score #accuracy
from sklearn.metrics import recall_score #sensibilidad


In [362]:
# Funcion para extraer bases desde gsheets
def extrae_drive(url):
    edit = '/edit#gid='
    export ='/export?format=xlsx&gid='
    url = (url).replace(edit, export)
    return url

## Generando la Base principal
1. Se usara la base de **cesados** y la base **planilla**, esta sera filtrada segun las condiciones de la conclusion en el notebook previo. En esta nueva base se empezara a agregar el resto de informacion para el proyecto
    * Cesados para poder filtrar las conciones 
    * Planilla para poder obtener todos los colaboradores en el tiempo

In [430]:
# Extrae base de Cesados y planilla
cesados = extrae_drive('https://docs.google.com/spreadsheets/d/1bO_T0lAQgjcpjFFNVAwiXd2obwRjJS3K/edit#gid=2050623625')
cesados = pd.read_excel(
                cesados,
                usecols='E,I,O,P,S,T,W,X'
            ).rename(
                columns = {
                    'Total o Temprana':'TIEMPO_CESE',
                    'Producto':'PRODUCTO',
                    'TIPO':'TIPO_CESE',
                    'MOTIVO PRIMARIO':'MOTIVO_CESE'}
            )

planilla = extrae_drive('https://docs.google.com/spreadsheets/d/1L094YBwj3BvobbXb4t3ahzpmL6gBSxgs/edit#gid=648976010')
planilla = pd.read_excel(
                planilla,
                usecols='A,B,D,F'
            ).rename(
                columns={
                    'COD':'CODIGO'}
                    #'REGION/DPTO':'REGION',
                    #'DIVISION/GC':'DIVISION'    }
            )

2.  Al parecer hay colaboradores que cesaron mas de dos veces, es decir que ingresaron, cesaron, volvieron a ingresar y volvieron a cesar
    * Se tomara en cuenta este numero de ceses como una caracteristica (aunque son pocas las personas) ***Revisar**
    * Eliminare los colaboradores duplicados considerando solo los ultimos ceses, pues son de los que se tendria data mas reciente
3. Genero la columna antiguedad definida como CESE menos INGRESO en meses

In [431]:
# Colaboradores con mas de 2 ceses
cesados.groupby(by = 'CODIGO', as_index=False).agg(n = ('CODIGO','count')).query('n>=2')

,CODIGO,n
425,A22AAAAA34,2
535,A33AAAA149,2
580,A33AAAA21A,2
642,A34AAAAA55,2
688,A36AAAAA18,2
1151,A4AAAAA57A,2
1212,A4AAAAA742,2
1397,A57AAAAA17,2
1408,A57AAAAA37,2
1938,A84AAAAA56,2


In [432]:
# Obtener solo ultimos cesados
cesados = cesados.sort_values(
    by = 'CESE',
    ascending = False # de mayor a menor
).drop_duplicates(
    'CODIGO',
    keep = 'first' # los mas recientes
).iloc[:, [0,2,3,6,7]] #solo selecciono las columnas que usare

# Halla la antiguedad de los cesados en meses
cesados['antiguedad_meses'] = (((cesados['CESE'] - cesados['INGRESO']).dt.days)/30).round(0) # Creo antiguedad

4. Utilizo como base Planilla que tiene un registro de colabores cesados y activos
    * Elimino los duplicados pues, la planilla cuenta con registro de colaboradores por meses, me quedo solo con los mas recientes
    * Realizo los filtros para generar la base que me interesa
5. Creo la base con planilla y cesados

In [433]:
# Quito duplicados de planilla obtener solo un codigo por colaborador
planilla = planilla.drop_duplicates(
                'CODIGO',
                keep = 'last').iloc[:, 1:6] # no me interesa la fecha, solo los valores unicos del codigo

# CRedo df uniendo planilla y cesados                
df = pd.merge(
    planilla,
    cesados,
    on = 'CODIGO',
    how = 'left'
).query('PRODUCTO == "GRUPAL"')

6. Empiezo a generar los filtros identificados en el analisis anterios
    * Genero una base para Grupal y otra para invidual porque seran dos diferentes proyectos con caracteristicas diferentes
    * Empiezo a filtrar, Cargo, Tipo de cese, motivo y antiguedad
    * Como uni la base de ceses con planilla se han generado vacios de los colaboradores activos, para ellos debo de tener cuidado en los filtros

In [434]:
# Realizando filtros necesarios
df = df[
    (df['CARGO'] == 'VENTA GRUPAL')
    & (df['TIPO_CESE'] != 'INVOLUNTARIA') #vacio o voluntaria
    & (~df['MOTIVO_CESE'].isin(['Personal Eventual', 'No le Gusto el Trabajo'])) # los otros motivos o vacios
    & ((df['antiguedad_meses'] >= 9) # quito los mayores a 9 meses
    | (df['antiguedad_meses'].isna()))
    ] # o que sean activos

7. Creo la columna dependiente que quiero predecir 
    * Creo una nueva columna que clasifique si es activo o cesado, usando el TIPO_CESE como referencia
    * Reemplazo la fecha de los que no cesaron con la fecha de cierre, es decir el 01/01/2022
    *  Selecciono solo las variables que usare:
        * Ya no usare INGRESO porque necesito extraerno de otra base donde tenga el ingreso de los activos y cesados
        * Ya no usare TIPO_CESE,  MOTIVO_CESE ya no competen en el analisis
        * Ya no usare antiguedad_meses, pues generare una nueva

In [435]:
# Creando columna de variable dependiente
df.loc[df['TIPO_CESE'] == 'VOLUNTARIA', 'Estado'] = 'CESADO'
df.loc[df['TIPO_CESE'] != 'VOLUNTARIA', 'Estado'] = 'ACTIVO'
df.loc[df['CESE'].isna(), 'CESE'] = '2022-01-01'

# Seleccionnado variables
df = df.iloc[:,[0,4,8]]

# Base final antes de joins
df.sample(3)

,CODIGO,CESE,Estado
5515,113AAAAA36,2022-01-01,ACTIVO
5135,A74AAAAA39,2022-01-01,ACTIVO
5763,A79AAAA123,2022-01-01,ACTIVO


8. Es necesario ver si la base con la que vamos a continuar esta balanceada
    *  Como se muestra tenemos **35% de Cesados y 65% de Activos**, por lo que podemos continuar con el analisis

In [436]:
df.groupby(
    by = 'Estado',
    as_index = False
).agg(
    n = ('CODIGO' , 'count')
)['n'] / df.shape[0]

0    0.641563
1    0.358437
Name: n, dtype: float64

## Agregando variables a la base principal

### Datos personales
1. Agregar una tabla con caracteristicias principales e importantes de los colaboradores
    * Como se vio en los casos anteriores hay colaboradores con varias fechas de ingreso, es necesario hacer una limpieza de esto considerando solo los ultimos ingresos
    * Cambio a inicio de mes las fechas de ingreso para tener un estandar

In [437]:
datos_personales = extrae_drive('https://docs.google.com/spreadsheets/d/1ku5CfOhQBpML0gXkuRIIEDjqG0liPdKt/edit#gid=254224400')
datos_personales = pd.read_excel(
                        datos_personales,
                        usecols = 'A, C:G'
                    ).sort_values(
                        by = 'INGRESO',
                        ascending = False
                    ).drop_duplicates(
                        'CODIGO',
                        keep = 'first')

## Obteniendo la fecha inicial del mes en el ingreso    
datos_personales['INGRESO'] = datos_personales['INGRESO'].to_numpy().astype('datetime64[M]')

2. Agrero la tabla de los datos personales con la df que se tiene hasta el momento
3. Hallo la antiguedad y la edad en la que cesaron con el nuevo cruce


In [438]:
# Uniendo datos personales a df
df = pd.merge(
        df,
        datos_personales,
        on = 'CODIGO',
        how = 'left')
        
# Hallando la antiguedad en la Fecha de cese y fecha ingreso
df['antiguedad'] = (((df['CESE'] - df['INGRESO']).dt.days)/30).round(0) #meses
df['edad'] = (((df['CESE'] - df['NACIMIENTO']).dt.days)/360).round(0) #anios

4. Limpieando antiguedades erroneas
    * Encontre negativos en la diferencia de edad, esto debido a que hay personas que tienen una fecha de cese que es de su primer ingreso, y una fecha de segundo ingreso, pero a la fecha no cesan por segunda vez. Para solucionar esto, debo reemplazar la fecha cese por la fecha de cierre que estoy tomando (enero 2021) y pasar de Estado CESADO a ACTIVO.
    * Recalculo la columna antiguedad  para tener las nuevas antiguedades con los cambios de fecha


In [439]:
# Colaboradores con antiguedad errada
df.query('antiguedad < 0').sort_values(by = 'antiguedad')['CODIGO'].unique()

# Limpiando las antiguedades erradas
cod = df.query('antiguedad < 0')['CODIGO'].unique()
df.loc[df['CODIGO'].isin(cod), 'CESE'] = '2021-01-01'
df.loc[df['CODIGO'].isin(cod), 'Estado'] = 'ACTIVO'
df['antiguedad'] = (((df['CESE'] - df['INGRESO']).dt.days)/30).round(0) #meses

In [440]:
df.info() #sin valores nulos

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2098 entries, 0 to 2097
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   CODIGO      2098 non-null   object        
 1   CESE        2098 non-null   datetime64[ns]
 2   Estado      2098 non-null   object        
 3   INGRESO     2098 non-null   datetime64[ns]
 4   NACIMIENTO  2098 non-null   datetime64[ns]
 5   SEXO        2098 non-null   object        
 6   Educacion   2098 non-null   object        
 7   EC          2098 non-null   object        
 8   antiguedad  2098 non-null   float64       
 9   edad        2098 non-null   float64       
dtypes: datetime64[ns](3), float64(2), object(5)
memory usage: 180.3+ KB


### Bonos de Venta Grupal
* Este valor diferencia el ingreso extra que tiene un colaborador de otro respecto a su desempeno, lo llamaremos RV por Remuneracion Variable


In [441]:
bonos_venta = extrae_drive('https://docs.google.com/spreadsheets/d/1POTE8j5SneZ0QSvmbtKL_t4hu2odWlTM/edit#gid=578896078')
bonos_venta = pd.read_excel(
                    bonos_venta,
                    usecols = 'A, B, CE'
                ).rename(
                    columns = {
                    'Codigo Trabajador': 'CODIGO',
                    'RV TOTAL': 'RV'}
                )

1. Analizo el comportamiento del RV en el tiempo
    * Como se ve, los periodos de marzo 2020 a octubre 2020 (inclusive) tienen valores muy bajos debido a la pandemia, por lo que para este analisis no lo tomaremos en cuenta, pues nos bajaria el promedio de maner atipica


In [442]:
# graficando rv historico incluido periodo covid
rv_agrup = bonos_venta.groupby(
                by = 'MES',
                as_index = False
            ).agg(
                rv_total = ('RV' , 'sum')
            )

principal = alt.Chart(
    rv_agrup
).mark_line(
    color = '#04328C'
).encode(
    x = alt.X(
        'MES:T',
        title = None),
    y = alt.Y(
        'rv_total:Q',
        title = None,
        axis = None)
)

linea_i = alt.Chart(
    pd.DataFrame({'MES': ['2020-03-01']})
).mark_rule(
    color = 'red',
    strokeWidth = 2
).encode(
    x = 'MES:T')

linea_f = alt.Chart(
    pd.DataFrame({'MES': ['2020-10-01']})
).mark_rule(
    color = 'red',
    strokeWidth = 2
).encode(
    x = 'MES:T')


(principal + linea_i + linea_f).properties(
    title = 'RV total Historico',
    width = 700,
    height = 300
).configure_view(
    strokeWidth = 0
 ).configure_axis(
    labelFontSize = 13,
    labelAngle = 0
 ).configure_title(
    fontSize = 18,
    font = 'courier',
    color = '#033E8C',
    anchor = 'start'
)

alt.LayerChart(...)

2. RV promedio historico sin covid
    * Filtro a la base RV los meses de atipicos de covid
    * Utilizaremos el rv promedio y la media pues estos varian mucho en algunos casos
3. Agrero al df la nueva base de RV

In [443]:
# Filtrando periodo covid
bonos_venta = bonos_venta.query(
                'MES <= "2020-03-01" | MES >= "2020-10-01" '
            ).groupby(
                by = 'CODIGO',
                as_index = False
            ).agg(
                rv_prom = ('RV', 'mean'),
                rv_median = ('RV', 'median')
            )

# uniendo a df
df = pd.merge(
        df,
        bonos_venta,
        on = 'CODIGO',
        how = 'left')

4. Valores faltantes de RV 
    * Se han encontrado 13 colaboradores que no estan en la base del RV, al parecer porque son reciente y no han generado un bono extra
    * Debido a que son pocos, se reemplazaran los null con el promedio del RV total

In [444]:
# Verificando la cantidad nas
df.isna().sum()

# Reemplazando nas por media y promedio
df['rv_prom'] = df['rv_prom'].fillna(bonos_venta['rv_prom'].mean())
df['rv_median'] = df['rv_median'].fillna(bonos_venta['rv_median'].mean())

### Programa de Mejora
* Analizando los colaboradores del cargo Venta Grupal que estuvieron en el programa de mejora durante todo su periodo en la empresa
* El programa de mejora tiene el objetivo que entrenar al colaborador durante unos 3 meses aprox para mejorar su disciplina o desempeno

In [445]:
programa_mejora = extrae_drive('https://docs.google.com/spreadsheets/d/1CKFlLpx248yvZS0INAawAA5e8NC2qdvV/edit#gid=1510069928')
programa_mejora = pd.read_excel(
                    programa_mejora,
                    usecols = 'C, F, Q'
                ).rename(
                    columns = {
                    'CODIGO CFIS': 'CODIGO',
                    'MOTIVO PRINCIPAL': 'MOTIVO_PM'}
                ).query(
                    'Cargo == "VENTA GRUPAL"'
                )

1. Hallo el numero de veces que el colaborador estuvo en el programa de mejoras
    * Al parecer hay algunos colaboradores que tienen mas de 3 veces en el programa
    * El numero de veces que estuvo en el programa de mejora segun el motivo
2. Uno as bases del programa de mejora al df general

In [446]:
# Nro de veces en el programa
pm_total = programa_mejora.groupby(
                    by='CODIGO', as_index=False
                ).agg(
                    pm_total = ('CODIGO','count')
                )
                
# Nro de veces en el programa segun el motivo
pm_motivos = pd.get_dummies(
                programa_mejora,
                columns = ['MOTIVO_PM']
            ).groupby(
                by = 'CODIGO', as_index = False
            ).agg(
                MOTIVO_PM_DESEMP = ('MOTIVO_PM_DESEMPEÑO','sum'),
                MOTIVO_PM_DISCIPLINA = ('MOTIVO_PM_DISCIPLINA','sum')
            )

In [447]:
# Uniendo con total de programas
df = pd.merge(
        df,
        pm_total,
        on = 'CODIGO',
        how = 'left')

# Uniendo con motivos
df = pd.merge(
        df,
        pm_motivos,
        on = 'CODIGO',
        how = 'left')

### Memos aplicados
* Dentro de la empresa se aplican memos cuando existe algun tipo de falta, para este caso sabre cuantos memos tienen los colaboradores
1. Cuento los memos por colaborador y los uno al df

In [448]:
memos = extrae_drive('https://docs.google.com/spreadsheets/d/1Q2pvICQWLAh8Rvou9FvH085KULXioDVK/edit#gid=238204385')
memos = pd.read_excel(
                    memos,
                    usecols = 'B, D'
                ).rename(
                    columns = {
                    'CÓDIGO': 'CODIGO'}
                ).query(
                    'CARGO == "VENTA GRUPAL"'
                )

In [449]:
# Memor por colaborador
memos = memos.groupby(
                by = 'CODIGO',
                as_index = False
            ).agg(
                memos_total = ('CODIGO','count')
            )

# uniendo total de memos al df             
df = pd.merge(
        df,
        memos,
        on = 'CODIGO',
        how = 'left')

### Total de Hijxs
* Colaboradores con nro total de hijos, como no se saber que hijos tien a su cargo o no, se esta tomando hijos con edad menor a 18 

In [450]:
hijxs = extrae_drive('https://docs.google.com/spreadsheets/d/16lmjxH0f9sav7thbqeWGrdEsckr9Ia_u/edit#gid=1468377625')
hijxs = pd.read_excel(
                    hijxs,
                    usecols = 'D, F'
                ).rename(
                    columns = {'COD CFIS': 'CODIGO'}
                )

1. Hallando la edad del hijx a la fecha de cese
    * Solo tomare en cuenta a los hijxs con edad menor o igual a 18 en la fecha de cese
    * Para esto necesito optener la fecha de cese y hallar la edad
2. Filtrar la edad menor a 18 y contar el numero de hijxs
3. Unir la cantidad de hijxs con la base df 

In [451]:
# optener la fecha de cese
hijxs = pd.merge(
        hijxs,
        df[['CODIGO','CESE']],
        on = 'CODIGO',
        how = 'left')

# Halla la edad en el cese
hijxs['edad'] = (((hijxs['CESE'] - hijxs['FECHA_NACIMIENTO']).dt.days)/360).round(0)

# Total hijos menores a 18
hijxs = hijxs.query(
                'edad <= 18'
            ).groupby(
                by = 'CODIGO',
                as_index = False
            ).agg(
                hijxs = ('CODIGO','count')     
            )
            
# Uniendo hijos con df
df = pd.merge(
        df,
        hijxs,
        on = 'CODIGO',
        how = 'left')

## Base de datos final
1. Rellenar datos faltantes

In [452]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2098 entries, 0 to 2097
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   CODIGO                2098 non-null   object        
 1   CESE                  2098 non-null   datetime64[ns]
 2   Estado                2098 non-null   object        
 3   INGRESO               2098 non-null   datetime64[ns]
 4   NACIMIENTO            2098 non-null   datetime64[ns]
 5   SEXO                  2098 non-null   object        
 6   Educacion             2098 non-null   object        
 7   EC                    2098 non-null   object        
 8   antiguedad            2098 non-null   float64       
 9   edad                  2098 non-null   float64       
 10  rv_prom               2098 non-null   float64       
 11  rv_median             2098 non-null   float64       
 12  pm_total              108 non-null    float64       
 13  MOTIVO_PM_DESEMP  

In [453]:
# Reemplazando pm_total y motivos del programa de mejora vacios por 0

df['pm_total'] = df['pm_total'].fillna(0)
df['MOTIVO_PM_DESEMP'] = df['MOTIVO_PM_DESEMP'].fillna(0)
df['MOTIVO_PM_DISCIPLINA'] = df['MOTIVO_PM_DISCIPLINA'].fillna(0)

# Reemplazando con 0 los memos vacios
df['memos_total'] = df['memos_total'].fillna(0)

# Reemplazando con 0 los colaboradores sin hijxs
df['hijxs'] = df['hijxs'].fillna(0)


2. Normalizar valores de Estado y Educacion

In [454]:
print('Sexo:', df['SEXO'].unique())
print('Estado Civil', df['EC'].unique())
print('Educacion', df['Educacion'].unique())

Sexo: ['M' 'F']
Estado Civil ['SOLTERO/A' 'CONCUBINATO / CONVIVIENTE' 'CASADO' 'SEPARADO' 'DIVORCIADO']
Educacion ['EDUCACIÓN UNIVERSITARIA COMPLETA' 'EDUCACIÓN UNIVERSITARIA INCOMPLETA'
 'GRADO DE BACHILLER' 'TITULADO' 'EDUCACIÓN TÉCNICA INCOMPLETA'
 'EDUCACIÓN TÉCNICA COMPLETA'
 'EDUCACIÓN SUPERIOR (INSTITUTO SUPERIOR, ETC) INCOMPLETA'
 'ESTUDIOS DE MAESTRÍA INCOMPLETA' 'EDUCACIÓN SECUNDARIA COMPLETA'
 'EDUCACIÓN SUPERIOR (INSTITUTO SUPERIOR, ETC) COMPLETA']


In [455]:
# Estandarizando Estado civil
df.loc[df['EC']=='CONCUBINATO / CONVIVIENTE', 'EC'] = 'CASADO'
df.loc[df['EC']=='SEPARADO', 'EC'] = 'DIVORCIADO'
df.loc[df['EC']=='SOLTERO/A', 'EC'] = 'SOLTERO'

# Estandarizando Educacion
df.loc[df['Educacion']=='EDUCACIÓN UNIVERSITARIA COMPLETA', 'Educacion'] = 'Univ comp'
df.loc[df['Educacion']=='EDUCACIÓN UNIVERSITARIA INCOMPLETA', 'Educacion'] = 'Univ inc'
df.loc[df['Educacion']=='GRADO DE BACHILLER', 'Educacion'] = 'Univ comp'
df.loc[df['Educacion']=='TITULADO', 'Educacion'] = 'Univ comp'
df.loc[df['Educacion']=='EDUCACIÓN TÉCNICA INCOMPLETA', 'Educacion'] = 'Tec inc'
df.loc[df['Educacion']=='EDUCACIÓN TÉCNICA COMPLETA', 'Educacion'] = 'Tec comp'
df.loc[df['Educacion']=='EDUCACIÓN SUPERIOR (INSTITUTO SUPERIOR, ETC) INCOMPLETA', 'Educacion'] = 'Tec inc'
df.loc[df['Educacion']=='EDUCACIÓN SUPERIOR (INSTITUTO SUPERIOR, ETC) COMPLETA', 'Educacion'] = 'Tec comp'
df.loc[df['Educacion']=='ESTUDIOS DE MAESTRÍA INCOMPLETA', 'Educacion'] = 'Maestria'
df.loc[df['Educacion']=='EDUCACIÓN SECUNDARIA COMPLETA', 'Educacion'] = 'Sec'

## Exploracion Grafica de Base Final
1. Diferencia del comportacion en Bono Variable Promedio y Mediana de Cesados y No Cesados
    * Se muestra que hay mayor RV promedio y RV median en los activos 
2. Diferencia del comportacion en Antiguedad y Edad de Cesados y No Cesados
    * Hay una diferencia de antiguedad donde los activos tiene mas tiempo que los cesados
    * En edad no existe mucha diferencia entre activos y cesados


In [463]:
# Comparacion Bonos
# rv promedio
g1 = alt.Chart(df).mark_boxplot(size=50, extent=0.5).encode(
        x = alt.X(
            'Estado:O',
            title = None),
        y = alt.Y(
            'rv_prom:Q',
            scale = alt.Scale(zero=False),
            title = 'Bono Variable Promedio'),
        ).properties(
            title = 'Rv Promedio Cesados vs Activos',
            width = 300,
            height = 300
        )

# rv mediana
g2 = alt.Chart(df).mark_boxplot(size=50, extent=0.5).encode(
        x = alt.X(
            'Estado:O',
            title = None),
        y = alt.Y(
            'rv_median:Q',
            scale = alt.Scale(zero=False),
            title = 'Bono Variable Mediana')
        ).properties(
            title = 'RV Mediana Cesados vs Activos',
            width = 300,
            height = 300
        )

## uniendo
alt.hconcat(g1, g2).configure_view(
    strokeWidth = 0
).configure_axis(
    labelFontSize = 13,
    labelAngle = 0
).configure_title(
    fontSize = 18,
    font = 'courier',
    color = '#033E8C',
    anchor = 'start'
)

alt.HConcatChart(...)

In [465]:
# Comparacion Antiguedad y Edad
# Antiguedad en meses
g1 = alt.Chart(df).mark_boxplot(size=50, extent=0.5).encode(
        x = alt.X(
            'Estado:O',
            title = None),
        y = alt.Y(
            'antiguedad:Q',
            scale = alt.Scale(zero=False),
            title = 'Antiguedad(meses)')
    ).properties(
        title = 'Antiguedad Cesados vs Activos',
        width = 300,
        height = 300
    )

# edad de cese
g2 = alt.Chart(df).mark_boxplot(size=50, extent=0.5).encode(
        x = alt.X(
            'Estado:O',
            title = None),
        y = alt.Y(
            'edad:Q',
            scale = alt.Scale(zero=False),
            title = 'Edad')
    ).properties(
        title = 'Edad Cesados vs Activos',
        width = 300,
        height = 300
    )

## uniendo
alt.hconcat(g1, g2).configure_view(
    strokeWidth = 0
).configure_axis(
    labelFontSize = 13,
    labelAngle = 0
).configure_title(
    fontSize = 18,
    font = 'courier',
    color = '#033E8C',
    anchor = 'start'
)

alt.HConcatChart(...)

## Preparacion para el modelado
1. Convertir a numericos los datos categoricos

In [391]:
# Transformando sexo y estado (acivo o cesado)
df = pd.get_dummies(df, columns = ['SEXO', 'Estado'], drop_first=True)

# Tranformando Estado Civil y Educacion
df = pd.get_dummies(df, columns = ['EC', 'Educacion'])


2. Base de datos final antes del Modelo ML

In [392]:
df = df.drop(columns=['CODIGO','CESE','INGRESO','NACIMIENTO'])

In [393]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2098 entries, 0 to 2097
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   antiguedad            2098 non-null   float64
 1   edad                  2098 non-null   float64
 2   rv_prom               2098 non-null   float64
 3   rv_median             2098 non-null   float64
 4   pm_total              2098 non-null   float64
 5   MOTIVO_PM_DESEMP      2098 non-null   float64
 6   MOTIVO_PM_DISCIPLINA  2098 non-null   float64
 7   memos_total           2098 non-null   float64
 8   hijxs                 2098 non-null   float64
 9   SEXO_M                2098 non-null   uint8  
 10  Estado_CESADO         2098 non-null   uint8  
 11  EC_CASADO             2098 non-null   uint8  
 12  EC_DIVORCIADO         2098 non-null   uint8  
 13  EC_SOLTERO            2098 non-null   uint8  
 14  Educacion_Maestria    2098 non-null   uint8  
 15  Educacion_Sec        

## Creacion del Modelo

0. Parto la base de datos en X Y separando mi variable dependediente de las variables independientes

In [394]:
x = df.drop(columns=['Estado_CESADO'])
y = df['Estado_CESADO']

1. Parto mi base en 3 para entrenar mi modelo

In [395]:
#Separo los datos de "train" en entrenamiento y prueba para probar los algoritmos
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

2. Escalando datos

In [396]:
escalador = StandardScaler()
x_train = escalador.fit_transform(x_train)
x_test = escalador.fit_transform(x_test)

3. Regresion logistica

In [397]:
reg_log = LogisticRegression().fit(x_train, y_train)
y_pred = reg_log.predict(x_test)

4. Validando el modelo
    * Para este caso es mas importante poder predecir quiene realmente se van, si tengo errores prediciendo a colaboradores que no se van no tendria tanto impacto negativo
    * Me importaria reducir los falsos negativos, por eso me indicador mas importante es el recall

In [398]:
matriz = confusion_matrix(y_test, y_pred)
matriz

array([[227,  42],
       [ 61,  90]], dtype=int64)

In [399]:
#accuracy
exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo: ', str(exactitud.round(2)))

# recall
sensibilidad = recall_score(y_test, y_pred)
print('sensibilidad del modelo: ', str(sensibilidad.round(2)))

Exactitud del modelo:  0.75
sensibilidad del modelo:  0.6
